In [1]:
# Import dependencies
import pandas as pd
from sklearn import linear_model
from sqlalchemy import create_engine

In [2]:
#Import functions
def model(df, position, var):
    df = df.dropna()
    df = df[df['pos'] == position]   

    # the model will use different independent variables depending on position
    if position == 'QB':
        X = df[var]
    elif position == 'RB':
        X = df[var]
    elif position == 'WR' or 'TE':
        X = df[var]
    else:
        print('Invalid position entered')
        return

    y = df['next_yr_ppg']
    reg = linear_model.LinearRegression()
    reg.fit(X, y)
    return reg

def testModelAccuracy(model, df, position, var):
    # creates a df from the csvFile, drops na values and rows where FantPos
    # does not equal the position parameter
    df = df.dropna()
    df = df[df['pos'] == position]

    # the model will use different independent variables depending on position
    if position == 'QB':
        XTest = df[var]
    elif position == 'RB':
        XTest = df[var]
    elif position == 'WR' or 'TE':
        XTest = df[var]
    else:
        print('Invalid position entered')
        return

    yTest = df['next_yr_ppg']
    results = model.score(XTest, yTest)
    return results

def testModelDifference(model, df, position, var):
    # creates a df from the csvFile, drops na values and rows where FantPos
    # does not equal the position parameter
    df = df.dropna()
    df = df[df['pos'] == position]

    # the model will use different independent variables depending on position
    if position == 'QB':
        XTest = df[var]
    elif position == 'RB':
        XTest = df[var]
    elif position == 'WR' or 'TE':
        XTest = df[var]
    else:
        print('Invalid position entered')
        return


    yPred = model.predict(XTest)
    predAndActual = {'Name': df['player'], 'Predicted PPG': yPred,
                     'Actual PPG': df['next_yr_ppg']}

    # creates df from dictionary above
    database = pd.DataFrame(predAndActual)

    # creates a difference column which depicts the difference between the
    # predicted PPG and actual PPG
    database['Predicted PPG'] = database['Predicted PPG'].round(decimals=3)
    database['Difference'] = database['Predicted PPG'] - database['Actual PPG']
    database['Difference'] = database['Difference'].round(decimals=3)
    database['AbsDifference'] = database['Difference'].abs()
    meanDiff = round(database['Difference'].mean(), 3)
    medianDiff = round(database['Difference'].median(), 3)
    meanAbsDiff = round(database['AbsDifference'].mean(), 3)
    medianAbsDiff = round(database['AbsDifference'].median(), 3)

    return database, meanDiff, medianDiff, meanAbsDiff, medianAbsDiff

def testModel(model, test, train, position, var):
    accuracy = testModelAccuracy(model, train, position, var)
    differences = testModelDifference(model, test, position, var)
    meanDiff = differences[1]
    medDiff = differences[2]
    meanAbsDiff = differences[3]
    medAbsDiff = differences[4]

    print('The accuracy of the {0} model is {1}'.format(position, accuracy))
    print('The {0} model has an average error of {1} PPG and an average absolute error of {2} PPG'.format(position, meanDiff, meanAbsDiff))
    print('The {0} model has a median error of {1} PPG and a median absolute error of {2} PPG'.format(position, medDiff, medAbsDiff))
    print('\n')
    return

def useModel(model, df1, df2, position, var):
    df1.dropna()
    df2.dropna()
    # checks where the Fant Pos is the position given and returns a data frame
    # with only the rows that include said position
    df1 = df1[df1['pos'] == position]
    df2 = df2[df2['pos'] == position]
    df2 = df2[['plid','player','ppr_g']]
    newdf = df1.merge(df2,how='inner',left_on='plid',right_on='plid')

    # the model will use difference parameters based on position
    if position == 'QB':
        X = df1[var]
    elif position == 'RB':
        X = df1[var]
    elif position == 'WR' or 'TE':
        X = df1[var]
    else:
        print('Invalid position entered')
        return
    yPred = model.predict(X)

    # creates new df with the name of player, their position, and their
    # predicted PPG
    databaseDict = {'Name': df1['player'], 'Pos': df1['pos'], 'Predicted PPG': yPred}
    database = pd.DataFrame(databaseDict)
    database = database.merge(df2,how='inner',left_on='Name',right_on='player')
    database = database.drop(columns=['player','plid'])
    database['Difference'] = database['Predicted PPG'] - database['ppr_g']
    database = database.sort_values(by = ['Predicted PPG'], ascending = False)
    database['Predicted PPG'] = database['Predicted PPG'].round(decimals = 3)
    database['Predicted PPR'] = 17 * database['Predicted PPG']
    database['Predicted PPR'] = database['Predicted PPR'].round(decimals = 3)
    database['AbsDifference'] = database['Difference'].abs()
    meanDiff = round(database['Difference'].mean(), 3)
    medianDiff = round(database['Difference'].median(), 3)
    meanAbsDiff = round(database['AbsDifference'].mean(), 3)
    medianAbsDiff = round(database['AbsDifference'].median(), 3)
    print('The {0} model has an average error of {1} PPG and an average absolute error of {2} PPG'.format(position, meanDiff, meanAbsDiff))
    print('The {0} model has a median error of {1} PPG and a median absolute error of {2} PPG'.format(position, medianDiff, medianAbsDiff))


     # this adds a position rank column to the dataframe
#     posRank = []
#     posRankNum = 1
#     for index, row in newdf.iterrows():
#         posRank.append(posRankNum)
#         posRankNum += 1

#     database['PosRank'] = posRank


    database = database.reset_index(drop = True)

    return database

In [3]:
# Connect to Database
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'fballfinalproject.c6sg90iemyn2.us-east-2.rds.amazonaws.com' ## INSERT YOUR DB ADDRESS 
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres' ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'FFForesight5!!' ## CHANGE THIS TO YOUR POSTGRES PASSWORD 
POSTGRES_DBNAME = 'postgres' ## CHANGE THIS TO YOUR DATABASE NAME
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}').format(
    username=POSTGRES_USERNAME,
    password=POSTGRES_PASSWORD,
    ipaddress=POSTGRES_ADDRESS,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DBNAME)
# Create the connection
cnx = create_engine(postgres_str)

/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
# Read in dataframes and merge
# fantasy = pd.read_sql_query('''SELECT * FROM fantasy;''', cnx)
# teams = pd.read_sql_query('''SELECT * FROM teams;''', cnx)
# advrush = pd.read_sql_query('''SELECT * FROM advrush;''', cnx)
# df = pd.merge(fantasy,teams[['tmid','passrate']],on='tmid', how='inner')
# df = pd.merge(df,advrush,on='plid',how='inner')
# df = df.drop(columns=['player_y', 'tm_y', 'pos_y',
#                       'age_y','G_y', 'gs_y','tmid_y'])
# df = df.rename(columns={'player_x':'player','tm_x':'tm','pos_x':'pos','age_x':'age','G_x':'G','gs_x':'gs','tmid_x':'tmid'})
# df
df = pd.read_sql_query('''SELECT fantasy.*,
teams.pasatt_g AS tm_passatt,
teams.passrate,teams.rushatt_g AS tm_rushatt
FROM fantasy 
INNER JOIN teams 
ON fantasy.tmid = teams.tmid;''',cnx)

In [5]:
df

,player,tm,pos,age,G,gs,plid,tmid,cmp_g,passatt_g,...,posrk,ovrank,yr,ppr_g,starter,next_yr_ppg,next_yr_starter,tm_passatt,passrate,tm_rushatt
0,AJBrown,TEN,WR,24,13,13,AJBrown2021,TEN2021,0.000,0.154,...,32,85,2021,13.915,0,NaN,NaN,31.471,90.0,32.412
1,AJBrown,TEN,WR,23,14,12,AJBrown2020,TEN2020,0.000,0.000,...,9,38,2020,17.679,0,13.915,0.0,30.312,105.9,32.562
2,AJBrown,TEN,WR,22,16,11,AJBrown2019,TEN2019,0.000,0.000,...,9,62,2019,13.569,0,17.679,0.0,28.000,108.3,27.812
3,AJDerby,MIA,TE,27,4,0,AJDerby2018,MIA2018,0.000,0.000,...,76,400,2018,3.450,0,NaN,NaN,28.438,93.0,23.188
4,AJDillon,GNB,RB,23,17,2,AJDillon2021,GNB2021,0.000,0.000,...,21,80,2021,10.918,0,NaN,NaN,34.882,107.4,26.235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5261,ZayJones,LVR,WR,25,16,2,ZayJones2020,LVR2020,0.062,0.062,...,130,321,2020,2.288,0,6.229,0.0,34.438,100.4,28.562
5262,ZayJones,BUF,WR,23,16,15,ZayJones2018,BUF2018,0.000,0.062,...,34,93,2018,10.325,0,3.260,0.0,31.188,62.6,29.250
5263,ZayJones,BUF,WR,22,15,10,ZayJones2017,BUF2017,0.000,0.000,...,90,218,2017,4.707,0,10.325,0.0,29.750,82.1,30.438
5264,ZurlonTipton,IND,RB,25,10,0,ZurlonTipton2015,IND2015,0.000,0.000,...,124,419,2015,1.270,0,NaN,NaN,38.688,77.5,24.750


In [6]:
# Manipulate df to add rush and target share
df['rushshare'] = df['rushatt_g'] / df['tm_rushatt']
df['targetshare'] = df['tgt_g'] / df['tm_passatt']

In [7]:
df = df.sort_values(by = ['player','yr'], ascending = [True,False])

In [8]:
nextYearTm = []
nextYearTmID = []
lastPlayer = 'NaN'
lastPlayerTm = 'NaN'


for index, row in df.iterrows():
    player = row['player']
    if lastPlayer == player:
        nextYearTm.append(lastPlayerTm)
        yr = row['yr'] + 1
        nextYearTmID.append(lastPlayerTm + str(yr))
    else:
        nextYearTm.append('NaN')
        nextYearTmID.append('NaN')
    lastPlayer = row['player']
    lastPlayerTm = row['tm']

df['NextTm'] = nextYearTm
df['NextTmID'] = nextYearTmID

In [9]:
starters = pd.read_csv('Database_CSVs/starters.csv')

In [10]:
import numpy as np
starters.rename({'PlID':'StID'},axis=1,inplace=True)
starters.rename({'Starter':'qbstarter'},axis=1,inplace=True)
df = pd.merge(df,starters[['TmID','qbstarter','StID']],left_on='NextTmID',right_on='TmID',how='outer')
df = df[df['player'].notna()]
df.drop(columns='TmID',inplace=True)
df['age'] = df['age'].apply(np.int64)
df['G'] = df['G'].apply(np.int64)
df['gs'] = df['gs'].apply(np.int64)
df['posrk'] = df['posrk'].apply(np.int64)
df['ovrank'] = df['ovrank'].apply(np.int64)
df['yr'] = df['yr'].apply(np.int64)
df['starter'] = df['starter'].apply(np.int64)

In [11]:
df = df.sort_values(by = ['player','yr'], ascending = [True,False],ignore_index=True)

In [12]:
connect = []
for index, row in df.iterrows():
    if row['qbstarter'] == 'NaN':
        connect.append('NaN')
    else:
        connect.append(str(row['qbstarter'])+str(row['yr']))
df['connect'] = connect

In [13]:
startpass = pd.read_csv('Database_CSVs/passerratings.csv')

In [14]:
startpass.rename({'PlID':'StartID'},axis=1,inplace=True)
startpass.rename({'AvgRate':'AveragePassRate'},axis=1,inplace=True)
startpass.rename({'PassRate':'OverallPassRate'},axis=1,inplace=True)
df = pd.merge(df,startpass[['StartID','AveragePassRate','OverallPassRate']],left_on='connect',right_on='StartID',how='outer')
df = df[df['player'].notna()]
df.drop(columns='StartID',inplace=True)
df['age'] = df['age'].apply(np.int64)
df['G'] = df['G'].apply(np.int64)
df['gs'] = df['gs'].apply(np.int64)
df['posrk'] = df['posrk'].apply(np.int64)
df['ovrank'] = df['ovrank'].apply(np.int64)
df['yr'] = df['yr'].apply(np.int64)
df['starter'] = df['starter'].apply(np.int64)

In [15]:
df = df.sort_values(by = ['player','yr'], ascending = [True,False],ignore_index=True)

In [16]:
# List out columns
df.columns

Index(['player', 'tm', 'pos', 'age', 'G', 'gs', 'plid', 'tmid', 'cmp_g',
       'passatt_g', 'passyds_g', 'passtd_g', 'int_g', 'rushatt_g', 'rushyds_g',
       'rushyds_att', 'rushtd_g', 'tgt_g', 'rec_g', 'recyds_g', 'yds_rec',
       'rectd_g', 'fmb_g', 'fl_g', 'tottd_g', '2PM_G', '2PP_G', 'fpts',
       'posrk', 'ovrank', 'yr', 'ppr_g', 'starter', 'next_yr_ppg',
       'next_yr_starter', 'tm_passatt', 'passrate', 'tm_rushatt', 'rushshare',
       'targetshare', 'NextTm', 'NextTmID', 'qbstarter', 'StID', 'connect',
       'AveragePassRate', 'OverallPassRate'],
      dtype='object')

In [17]:
# Select columns for each position (WR + TE joined in recvar)
qbvar = ['age','passatt_g','passyds_g','passtd_g','rushatt_g','rushyds_g',
         'rushyds_att','ppr_g','starter','next_yr_starter']
rbvar = ['age','rushatt_g','rushyds_g','rushyds_att','tgt_g','rec_g',
         'recyds_g','yds_rec','ppr_g','rushshare','targetshare']
recvar = ['age','tgt_g','rec_g','recyds_g','yds_rec','passrate','targetshare','OverallPassRate']

In [18]:
# Separate into training, testing, using, and actual data by Yr (Fantasy) or Year (other dataframes)
train = df[df['yr'] <= 2012]
test = df[(df['yr'] >= 2019) & (df['yr'] <= 2020)]
use = df[df['yr'] == 2020]
use = use[use['next_yr_starter'].notna()]
use = use.astype({"next_yr_starter": int})
use = use[use['OverallPassRate'].notna()]
use = use.astype({"OverallPassRate": int})
actuals = df[df['yr'] == 2021]

In [19]:
# Name model and put in position
QBmodel = model(train, "QB", qbvar)
RBmodel = model(train, "RB", rbvar)
WRmodel = model(train, "WR", recvar)
TEmodel = model(train, "TE", recvar)
testModel(QBmodel, test, train, "QB", qbvar)
testModel(RBmodel, test, train, "RB", rbvar)
testModel(WRmodel, test, train, "WR", recvar)
testModel(TEmodel, test, train, "TE", recvar)

The accuracy of the QB model is 0.7806590538367456
The QB model has an average error of -1.158 PPG and an average absolute error of 4.29 PPG
The QB model has a median error of -0.396 PPG and a median absolute error of 3.604 PPG


The accuracy of the RB model is 0.5436434451016764
The RB model has an average error of 0.175 PPG and an average absolute error of 2.899 PPG
The RB model has a median error of 0.538 PPG and a median absolute error of 2.301 PPG


The accuracy of the WR model is 0.6035349768263871
The WR model has an average error of 0.65 PPG and an average absolute error of 3.277 PPG
The WR model has a median error of 1.129 PPG and a median absolute error of 2.92 PPG


The accuracy of the TE model is 0.5887532892711258
The TE model has an average error of 0.142 PPG and an average absolute error of 2.028 PPG
The TE model has a median error of 0.483 PPG and a median absolute error of 1.525 PPG




In [20]:
# Use model to make predictions and check predictions
QBStats = useModel(QBmodel, use, actuals, 'QB', qbvar)
RBStats = useModel(RBmodel, use, actuals,'RB', rbvar)
WRStats = useModel(WRmodel, use, actuals, 'WR', recvar)
TEStats = useModel(TEmodel, use, actuals, 'TE', recvar)

The QB model has an average error of -0.649 PPG and an average absolute error of 4.053 PPG
The QB model has a median error of 0.24 PPG and a median absolute error of 3.227 PPG
The RB model has an average error of 0.005 PPG and an average absolute error of 2.897 PPG
The RB model has a median error of 0.485 PPG and a median absolute error of 2.301 PPG
The WR model has an average error of 1.157 PPG and an average absolute error of 3.328 PPG
The WR model has a median error of 1.635 PPG and a median absolute error of 2.956 PPG
The TE model has an average error of 0.115 PPG and an average absolute error of 2.069 PPG
The TE model has a median error of 0.5 PPG and a median absolute error of 1.535 PPG


In [21]:
#pd.set_option('display.max_rows', None)
QBStats

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
0,DakPrescott,QB,19.964,20.038,-0.073993,339.388,0.073993
1,PatrickMahomes,QB,17.945,21.276,-3.331243,305.065,3.331243
2,KylerMurray,QB,17.806,21.464,-3.657982,302.702,3.657982
3,JustinHerbert,QB,17.755,22.400,-4.644522,301.835,4.644522
4,DanielJones,QB,17.300,15.227,2.072922,294.100,2.072922
5,RyanTannehill,QB,17.092,15.788,1.304436,290.564,1.304436
6,JoeBurrow,QB,16.873,19.638,-2.765177,286.841,2.765177
7,RussellWilson,QB,16.852,17.343,-0.490582,286.484,0.490582
8,MattRyan,QB,16.786,13.112,3.673507,285.362,3.673507
9,TeddyBridgewater,QB,16.755,14.479,2.276067,284.835,2.276067


In [22]:
QBStats.sort_values(by=['AbsDifference'])

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
38,CaseKeenum,QB,4.067,4.100,-0.032890,69.139,0.032890
0,DakPrescott,QB,19.964,20.038,-0.073993,339.388,0.073993
7,RussellWilson,QB,16.852,17.343,-0.490582,286.484,0.490582
24,TuaTagovailoa,QB,14.428,13.915,0.512983,245.276,0.512983
47,ChadHenne,QB,1.374,0.825,0.548683,23.358,0.548683
48,BlaineGabbert,QB,1.088,0.333,0.755011,18.496,0.755011
25,JimmyGaroppolo,QB,14.331,15.167,-0.836026,243.627,0.836026
15,DerekCarr,QB,15.994,15.118,0.876079,271.898,0.876079
20,JaredGoff,QB,14.883,13.893,0.989905,253.011,0.989905
30,NickMullens,QB,8.900,9.900,-1.000397,151.300,1.000397


In [23]:
RBStats

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
0,ChristianMcCaffrey,RB,20.304,18.214,2.089669,345.168,2.089669
1,DalvinCook,RB,19.996,15.869,4.126635,339.932,4.126635
2,DerrickHenry,RB,17.604,24.162,-6.558293,299.268,6.558293
3,MylesGaskin,RB,16.643,10.212,6.430522,282.931,6.430522
4,AaronJones,RB,15.975,15.267,0.707578,271.575,0.707578
...,...,...,...,...,...,...,...
90,KeithSmith,RB,1.505,1.041,0.463962,25.585,0.463962
91,MichaelBurton,RB,1.429,0.919,0.509787,24.293,0.509787
92,QadreeOllison,RB,1.304,2.462,-1.158073,22.168,1.158073
93,AndyJanovich,RB,1.142,0.915,0.227094,19.414,0.227094


In [24]:
RBStats.sort_values(by=['AbsDifference'])

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
38,AlexanderMattison,RB,7.955,7.869,0.085672,135.235,0.085672
31,JamaalWilliams,RB,9.121,9.215,-0.094415,155.057,0.094415
8,NickChubb,RB,15.224,15.379,-0.154725,258.808,0.154725
93,AndyJanovich,RB,1.142,0.915,0.227094,19.414,0.227094
69,KeShawnVaughn,RB,3.292,3.050,0.241602,55.964,0.241602
...,...,...,...,...,...,...,...
65,DarrelWilliams,RB,3.818,11.529,-7.711118,64.906,7.711118
18,MarlonMack,RB,12.124,2.150,9.974389,206.108,9.974389
80,RashaadPenny,RB,2.139,12.170,-10.030928,36.363,10.030928
42,LeonardFournette,RB,7.655,18.257,-10.601762,130.135,10.601762


In [25]:
WRStats

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
0,JustinJefferson,WR,18.808,19.435,-0.627285,319.736,0.627285
1,TyreekHill,WR,18.736,17.441,1.294790,318.512,1.294790
2,DKMetcalf,WR,18.510,14.371,4.138913,314.670,4.138913
3,CalvinRidley,WR,18.373,14.220,4.153071,312.341,4.153071
4,DavanteAdams,WR,17.543,21.519,-3.976046,298.231,3.976046
...,...,...,...,...,...,...,...
120,CJBoard,WR,1.691,1.617,0.073704,28.747,0.073704
121,BrandonZylstra,WR,1.391,3.769,-2.378361,23.647,2.378361
122,JakeKumerow,WR,0.901,0.320,0.580938,15.317,0.580938
123,DiontaeSpencer,WR,0.089,-0.220,0.308708,1.513,0.308708


In [26]:
WRStats.sort_values(by=['AbsDifference'])

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
113,MackHollins,WR,3.616,3.547,0.068781,61.472,0.068781
120,CJBoard,WR,1.691,1.617,0.073704,28.747,0.073704
30,RussellGage,WR,11.558,11.643,-0.085282,196.486,0.085282
7,StefonDiggs,WR,16.676,16.794,-0.117576,283.492,0.117576
116,AshtonDulin,WR,3.167,3.029,0.138206,53.839,0.138206
...,...,...,...,...,...,...,...
48,PrestonWilliams,WR,9.980,1.725,8.255341,169.660,8.255341
43,DeeboSamuel,WR,10.293,21.188,-10.895372,174.981,10.895372
29,KekeCoutee,WR,11.979,0.750,11.229250,203.643,11.229250
8,WillFuller,WR,16.517,4.300,12.217421,280.789,12.217421


In [27]:
TEStats

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
0,TravisKelce,TE,17.192,16.425,0.766742,292.264,0.766742
1,DarrenWaller,TE,15.311,12.136,3.174869,260.287,3.174869
2,GeorgeKittle,TE,15.191,14.143,1.048278,258.247,1.048278
3,MarkAndrews,TE,13.130,17.712,-4.582144,223.210,4.582144
4,EvanEngram,TE,11.304,6.833,4.471042,192.168,4.471042
...,...,...,...,...,...,...,...
62,CharlieWoerner,TE,0.765,0.600,0.164827,13.005,0.164827
63,FosterMoreau,TE,0.761,5.018,-4.257192,12.937,4.257192
64,JohnnyMundt,TE,0.729,0.317,0.412380,12.393,0.412380
65,DominiqueDafney,TE,0.511,0.540,-0.028740,8.687,0.028740


In [28]:
TEStats.sort_values(by=['AbsDifference'])

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference
65,DominiqueDafney,TE,0.511,0.540,-0.028740,8.687,0.028740
62,CharlieWoerner,TE,0.765,0.600,0.164827,13.005,0.164827
45,PharaohBrown,TE,2.366,2.540,-0.174252,40.222,0.174252
20,RobertTonyan,TE,6.498,6.300,0.198417,110.466,0.198417
38,WillDissly,TE,3.114,3.340,-0.225593,52.938,0.225593
...,...,...,...,...,...,...,...
23,DawsonKnox,TE,5.842,10.940,-5.097825,99.314,5.097825
41,TylerConklin,TE,2.900,8.135,-5.235235,49.300,5.235235
15,OJHoward,TE,7.765,1.971,5.793834,132.005,5.793834
47,MaxxWilliams,TE,2.280,8.260,-5.980026,38.760,5.980026


In [30]:
def combineData(listOfDataFrames):
    # creates a df which contains the dataframes in the list which is passed in
    df = pd.concat(listOfDataFrames)
    df = df.sort_values(by = ['Predicted PPR'], ascending = False)

    # gives players an overall ranking
    rankings = []
    rank = 1
    for index, row in df.iterrows():
        rankings.append(rank)
        rank += 1

    df['Ovr Rank'] = rankings

    df = df.reset_index(drop = True)
    return df

In [31]:
data = [QBStats, RBStats, WRStats, TEStats]
fullData = combineData(data)
fullData.to_csv('predictedStats2021.csv')

In [33]:
pd.set_option('display.max_rows', None)
fullData

,Name,Pos,Predicted PPG,ppr_g,Difference,Predicted PPR,AbsDifference,Ovr Rank
0,ChristianMcCaffrey,RB,20.304,18.214,2.089669,345.168,2.089669,1
1,DalvinCook,RB,19.996,15.869,4.126635,339.932,4.126635,2
2,DakPrescott,QB,19.964,20.038,-0.073993,339.388,0.073993,3
3,JustinJefferson,WR,18.808,19.435,-0.627285,319.736,0.627285,4
4,TyreekHill,WR,18.736,17.441,1.294790,318.512,1.294790,5
5,DKMetcalf,WR,18.510,14.371,4.138913,314.670,4.138913,6
6,CalvinRidley,WR,18.373,14.220,4.153071,312.341,4.153071,7
7,PatrickMahomes,QB,17.945,21.276,-3.331243,305.065,3.331243,8
8,KylerMurray,QB,17.806,21.464,-3.657982,302.702,3.657982,9
9,JustinHerbert,QB,17.755,22.400,-4.644522,301.835,4.644522,10
